In [570]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils, plot_model
import matplotlib.pyplot as pl
from sklearn import metrics
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D
from keras.models import model_from_json
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import os
import itertools

eda_folder = "/content/drive/MyDrive/PPG/PPG"

train_ids = ['001', '005', '006', '009', '010', '011', '012', '013', '015', '021', '022', '023',
             '024', '026', '027', '028', '031', '032', '033', '036', '038', '041', '042', '044',
             '045', '047', '053', '054', '055', '060', '062', '063', '064', '066', '070', '072',
             '073', '075', '076', '077', '078']
test_ids = ['002', '003', '007', '017', '018', '019', '020', '025', '029', '030', '039', '043',
            '048', '052', '059', '065', '068', '074']

x_train = pd.DataFrame()
x_test = pd.DataFrame()
y_train = []
y_test = []

data_train = []
data_test = []

#scaler = StandardScaler()

for id in train_ids:
    for emotion in ['anger', 'disgust', 'fear', 'happy', 'neutral', 'sad']:
        file_path = f"{eda_folder}/train/{emotion}/{id}.csv"
        data = pd.read_csv(file_path, header=None)
        scaled_data = scaler.fit_transform(data)
        data_train.append(scaled_data)
        y_train.append(emotion)  

for id in test_ids:
    for emotion in ['anger', 'disgust', 'fear', 'happy', 'neutral', 'sad']:
        file_path = f"{eda_folder}/test/{emotion}/{id}.csv"
        data = pd.read_csv(file_path, header=None)
        scaled_data = scaler.transform(data)
        data_test.append(scaled_data)
        y_test.append(emotion)

In [ ]:
for i in range(len(train_ids)):
    for j in range(6):
        features = np.concatenate((np.array([train_ids[i]]), data_train[i][j].flatten(), np.array([j])), axis=None)
        x_train = x_train.append(pd.Series(features), ignore_index=True)

for i in range(len(test_ids)):
    for j in range(6):
        features = np.concatenate((np.array([test_ids[i]]), data_test[i][j].flatten(), np.array([j])), axis=None)
        x_test = x_test.append(pd.Series(features), ignore_index=True)

In [572]:
print(x_train.dtypes)
print(x_test.dtypes)

0    object
1    object
2    object
dtype: object
0    object
1    object
2    object
dtype: object


In [573]:
label_mapping = {'anger': 1, 'disgust': 2, 'fear': 3, 'happy': 4, 'neutral': 5, 'sad': 6}
y_train = [label_mapping[y] for y in y_train]
y_test = [label_mapping[y] for y in y_test]

In [574]:
y_train=np.array(y_train)
y_test=np.array(y_test)

def scalar_stand(x_train, x_test):
    # The standard deviation of the training set is used to standardize the training set and the test set
    data_all = np.vstack((x_train, x_test))    # The data is reduced to one-dimensional tiling
    scalar = preprocessing.StandardScaler().fit(data_all)  # Data standardization functions under the sklearn.preprcoessing package
    x_train = scalar.transform(x_train)   #Call the.transform function to normalize the data
    x_test = scalar.transform(x_test)
    return x_train, x_test

In [575]:
print(x_train)
print(x_test)
print(y_train)
print(y_test)

       0                      1  2
0    001     1.0176418675412744  0
1    001     1.0749047993704828  1
2    001     1.1127266123687871  2
3    001     1.1236843338916604  3
4    001      1.107424489051268  4
..   ...                    ... ..
241  078   -0.25338321290061105  1
242  078   -0.20591610607194183  2
243  078   -0.13670707268888216  3
244  078  -0.054552464716185384  4
245  078    0.02710423654152532  5

[246 rows x 3 columns]
       0                     1  2
0    002  0.012942998554462485  0
1    002  0.016729865956246856  1
2    002  0.018386620444527522  2
3    002  0.017203224381469904  3
4    002  0.012232960916627915  4
..   ...                   ... ..
103  074   0.26713647289923853  1
104  074    0.2427585140002516  2
105  074   0.21625044218776102  3
106  074    0.1838253900599823  4
107  074    0.1421698486403542  5

[108 rows x 3 columns]
[1 2 3 4 5 6 1 2 3 4 5 6 1 2 3 4 5 6 1 2 3 4 5 6 1 2 3 4 5 6 1 2 3 4 5 6 1
 2 3 4 5 6 1 2 3 4 5 6 1 2 3 4 5 6 1 2 3 4 5 6 1 

In [576]:
# Defined neural network
model = Sequential()
model.add(Embedding(max_features, 1000, input_length = len(x_train[1])))
model.add(Conv1D(64, 14, activation = 'relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(64, 14, activation = 'relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(6))
 
model.summary()
 
model.compile(optimizer = RMSprop(lr = 1e-4),
             loss = 'binary_crossentropy',
             metrics = ['acc'])
 
history = model.fit(x_train, y_train,
                   epochs = 20,
                   batch_size = 246,
                   validation_split = 0.2)

Model: "sequential_119"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_27 (Embedding)    (None, 246, 1000)         40000000  
                                                                 
 conv1d_273 (Conv1D)         (None, 233, 64)           896064    
                                                                 
 max_pooling1d_118 (MaxPooli  (None, 46, 64)           0         
 ng1D)                                                           
                                                                 
 conv1d_274 (Conv1D)         (None, 33, 64)            57408     
                                                                 
 global_max_pooling1d_22 (Gl  (None, 64)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_83 (Dense)            (None, 6)              

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


ValueError: ignored